In [1]:
import requests
import nba_api
from nba_api.stats.endpoints import (playbyplayv2,playbyplay,leaguehustlestatsplayer,
                                     leaguedashplayerbiostats,commonplayerinfo,
                                     leaguestandings,playerdashptreb,
                                    playerdashboardbyshootingsplits)
import pandas as pd
import numpy as np
import time

In [2]:
# this method takes 2 minutes to append all dataframes (down from 12 minutes)
print(time.strftime("%H:%M:%S",time.localtime()))
seasons=list(range(1997,2022))
season_slug=[str(x)+'-'+str(x+1)[-2:] for x in seasons]
pbp=pd.read_csv(filepath_or_buffer="PBP Data/1996-97_pbp.csv")
for seas in season_slug:
    pbp=pbp.append(pd.read_csv(filepath_or_buffer="PBP Data/"+seas+"_pbp.csv"))
pbp=pbp.drop(columns=['VIDEO_AVAILABLE_FLAG','PLAYER3_TEAM_NICKNAME','PLAYER3_TEAM_CITY','PLAYER3_TEAM_ID',
                      'PLAYER2_TEAM_NICKNAME','PLAYER2_TEAM_CITY','PLAYER2_TEAM_ID','PLAYER1_TEAM_NICKNAME',
                      'PLAYER1_TEAM_CITY','PLAYER1_TEAM_ID','PERSON1TYPE','PERSON2TYPE','PERSON3TYPE','Unnamed: 0'],errors='ignore')
print(time.strftime("%H:%M:%S",time.localtime()))

15:11:07
15:12:57


In [3]:
# deleted because taking up too much space
# print(time.strftime("%H:%M:%S",time.localtime()))
# pbp=pd.read_csv(filepath_or_buffer="PBP Data/1997-2020_pbp.csv")
# print(time.strftime("%H:%M:%S",time.localtime()))

In [4]:
pbp['season']=pbp['GAME_ID'].astype(str).str[1:3].astype(int).add(1)

In [5]:
pbp

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,season
0,29600001,1,12,0,1,11:15 PM,12:00,NaN,Start of 1st Period (11:15 PM EST),NaN,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,97
1,29600001,2,10,0,1,11:16 PM,12:00,Jump Ball Ellison vs. Longley: Tip to Harper,NaN,NaN,...,442,Pervis Ellison,BOS,26,Luc Longley,CHI,166,Ron Harper,CHI,97
2,29600001,4,1,5,1,11:16 PM,11:39,NaN,NaN,Rodman Layup (2 PTS) (Longley 1 AST),...,23,Dennis Rodman,CHI,26,Luc Longley,CHI,0,NaN,BOS,97
3,29600001,5,6,2,1,11:16 PM,11:39,Ellison S.FOUL (P1.T1),NaN,NaN,...,442,Pervis Ellison,BOS,0,NaN,NaN,0,NaN,NaN,97
4,29600001,6,3,10,1,11:17 PM,11:39,NaN,NaN,Rodman Free Throw 1 of 1 (3 PTS),...,23,Dennis Rodman,CHI,0,NaN,NaN,0,NaN,NaN,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570251,22101230,670,2,1,4,12:03 AM,0:21,NaN,NaN,MISS Alexander-Walker 3PT Jump Shot,...,1629638,Nickeil Alexander-Walker,UTA,0,NaN,NaN,0,NaN,NaN,22
570252,22101230,671,4,0,4,12:03 AM,0:18,Perry REBOUND (Off:2 Def:6),NaN,NaN,...,1629617,Reggie Perry,POR,0,NaN,NaN,0,NaN,NaN,22
570253,22101230,672,2,1,4,12:03 AM,0:11,MISS Blevins 3PT Jump Shot,NaN,NaN,...,1629833,Keljin Blevins,POR,0,NaN,NaN,0,NaN,NaN,22
570254,22101230,673,4,0,4,12:03 AM,0:07,NaN,NaN,Sneed REBOUND (Off:1 Def:1),...,1630270,Xavier Sneed,UTA,0,NaN,NaN,0,NaN,NaN,22


# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [6]:
alley_oops=pbp[pbp['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp['VISITORDESCRIPTION'].str.contains('Alley',na=False)]

made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)].\
drop(columns=['PLAYER3_ID','PLAYER3_NAME','PLAYER3_TEAM_ABBREVIATION'])

alley_oops_w_2_players=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
alley_oops_w_2_players['p1']=alley_oops_w_2_players[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
alley_oops_w_2_players['p2']=alley_oops_w_2_players[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)

In [7]:
alley_oops_w_2_players.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(n=10,columns="count",keep="all")

,p1,p2,count
2783,Clint Capela,James Harden,360
2615,Chris Paul,DeAndre Jordan,258
1520,Blake Griffin,DeAndre Jordan,199
2649,Chris Paul,Tyson Chandler,197
2794,Clint Capela,Trae Young,161
604,Andre Drummond,Reggie Jackson,158
4477,Dwight Howard,Jameer Nelson,157
996,Anthony Davis,Jrue Holiday,139
4550,Dwight Powell,Luka Doncic,138
6740,John Collins,Trae Young,138


In [8]:
alley_oops_w_2_players.groupby(['season','p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(n=10,columns="count",keep="all")

,season,p1,p2,count
2971,8,Chris Paul,Tyson Chandler,107
9331,19,Clint Capela,James Harden,100
8560,18,Clint Capela,James Harden,98
11686,22,Clint Capela,Trae Young,92
7901,17,Clint Capela,James Harden,86
10852,21,Clint Capela,Trae Young,69
7263,16,Chris Paul,DeAndre Jordan,68
3437,9,Chris Paul,Tyson Chandler,65
11856,22,Dwight Powell,Luka Doncic,64
8461,18,Anthony Davis,Rajon Rondo,60


# The “What Goes Up...” Award (sponsored by the Isaac Newton Trust)

Most prolific alley-oop passers

In [9]:
alley_oops_w_2_players.groupby('PLAYER2_NAME').size().reset_index().rename(columns={0:'oops_started','PLAYER2_NAME':'player'}).nlargest(n=10,columns="oops_started",keep="all")

,player,oops_started
214,Chris Paul,796
48,Andre Miller,738
565,James Harden,709
1091,Russell Westbrook,543
588,Jason Kidd,488
1025,Rajon Rondo,474
383,Dwyane Wade,433
1231,Trae Young,409
807,LeBron James,394
653,John Wall,345


In [10]:
alley_oops_w_2_players.groupby(['season','PLAYER2_NAME']).size().reset_index().rename(columns={0:'oops_started','PLAYER2_NAME':'player'}).nlargest(n=10,columns="oops_started",keep="all")

,season,player,oops_started
5305,22,Trae Young,147
1467,8,Chris Paul,125
3657,17,James Harden,118
4980,21,Trae Young,115
4247,19,James Harden,113
5217,22,Luka Doncic,113
1026,6,Andre Miller,111
3940,18,James Harden,109
811,5,Andre Miller,93
4677,20,Spencer Dinwiddie,92


# The “...Must Come Down!” Award (sponsored by the US Apple Association)

Most prolific alley-oop finishers

In [11]:
alley_oops_w_2_players.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'oops_finished','PLAYER1_NAME':'player'}).nlargest(n=10,columns="oops_finished",keep="all")

,player,oops_finished
287,DeAndre Jordan,1126
358,Dwight Howard,1022
1189,Tyson Chandler,820
215,Clint Capela,652
45,Andre Drummond,612
66,Anthony Davis,583
488,JaVale McGee,561
1022,Rudy Gobert,506
748,LeBron James,445
690,Kenyon Martin,394


In [12]:
alley_oops_w_2_players.groupby(['season','PLAYER1_NAME']).size().reset_index().rename(columns={0:'oops_finished','PLAYER1_NAME':'player'}).nlargest(n=10,columns="oops_finished",keep="all")

,season,player,oops_finished
1415,8,Tyson Chandler,135
2584,14,DeAndre Jordan,133
3032,16,DeAndre Jordan,133
3472,18,Anthony Davis,129
2801,15,DeAndre Jordan,120
4579,22,Clint Capela,119
3771,19,Clint Capela,118
3493,18,Clint Capela,115
3266,17,DeAndre Jordan,113
3512,18,DeAndre Jordan,112


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop_soup for the idea and asw7412 for the sponsor)

In [13]:
misses=pbp[pbp.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]

In [14]:
#as the blockee
blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'times_blocked','PLAYER1_NAME':'player'}).nlargest(n=10,columns="times_blocked",keep="all")

,player,times_blocked
1015,Rudy Gobert,61
1151,Tristan Thompson,49
488,Ivica Zubac,45
287,DeAndre Jordan,42
314,Derrick Favors,42
926,Pau Gasol,42
1014,Rudy Gay,42
357,Dwight Howard,41
1123,Tim Duncan,40
857,Montrezl Harrell,38


In [15]:
blocked_dunks.groupby(['season','PLAYER1_NAME']).size().reset_index().rename(columns={0:'times_blocked','PLAYER1_NAME':'player'}).nlargest(n=10,columns="times_blocked",keep="all")

,season,player,times_blocked
2931,19,Rudy Gobert,15
1886,13,Tristan Thompson,13
1019,8,Emeka Okafor,12
3257,21,Ivica Zubac,12
2762,19,Andre Drummond,11
2830,19,Ivica Zubac,11
3551,22,Kenyon Martin Jr.,11
3607,22,Precious Achiuwa,11
316,3,Antonio Davis,10
1663,12,Kris Humphries,10


# The No Fly Zone Award*

most blocked dunks as the blocking player

In [16]:
#as the blocker
blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'times_blocker','PLAYER3_NAME':'player'}).nlargest(n=10,columns="times_blocker",keep="all")

,player,times_blocker
31,Alonzo Mourning,126
905,Serge Ibaka,123
285,Dikembe Mutombo,110
966,Tim Duncan,91
961,Theo Ratliff,90
518,Joel Przybilla,88
499,Jermaine O'Neal,87
329,Emeka Okafor,81
39,Andre Drummond,76
117,Brook Lopez,71


In [17]:
blocked_dunks.groupby(['season','PLAYER3_NAME']).size().reset_index().rename(columns={0:'times_blocker','PLAYER3_NAME':'player'}).nlargest(n=10,columns="times_blocker",keep="all")

,season,player,times_blocker
119,2,Alonzo Mourning,27
542,6,Alonzo Mourning,24
3290,100,Dikembe Mutombo,23
3271,100,Alonzo Mourning,22
485,5,Joel Przybilla,21
1877,16,Serge Ibaka,21
2226,19,Andre Drummond,21
2281,19,JaVale McGee,20
2744,21,Nerlens Noel,20
425,4,Theo Ratliff,18


In [18]:
# dunk_blocks=blocker.merge(blockee,on='player',how='outer').fillna(0)
# dunk_blocks['net_blocked_dunks']=dunk_blocks[['times_blocker']].subtract(dunk_blocks.times_blocked,axis=0)
# dunk_blocks['blocked_dunks_involved']=dunk_blocks[['times_blocker']].add(dunk_blocks.times_blocked,axis=0)
# dunk_blocks.sort_values('net_blocked_dunks',ascending=False).tail(10)
# dunk_blocks.sort_values('net_blocked_dunks',ascending=False).head(10)
# dunk_blocks.query('times_blocker == 0').sort_values('net_blocked_dunks',ascending=False).tail(10)

# The Bowling Ball Award (sponsored by [Pete Weber](https://www.youtube.com/watch?v=gKQOXYB2cd8))*

most charges committed (credit to Kdog122025 for the idea) (charges were first recorded starting in 2011)

In [19]:
charges=pbp[pbp['HOMEDESCRIPTION'].str.contains('Charge',na=False)|
        pbp['VISITORDESCRIPTION'].str.contains('Charge',na=False)]

In [20]:
charges.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
363,Giannis Antetokounmpo,210
887,Russell Westbrook,208
245,DeMarcus Cousins,186
453,James Harden,185
661,LeBron James,134
574,Julius Randle,123
823,Paul George,106
1024,Tyreke Evans,106
824,Paul Millsap,105
530,John Wall,102


In [21]:
charges.groupby(['season','PLAYER1_NAME']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,season,PLAYER1_NAME,count
2816,19,Giannis Antetokounmpo,41
286,11,Russell Westbrook,39
3161,20,Giannis Antetokounmpo,34
2406,18,DeMarcus Cousins,30
2513,18,Julius Randle,30
89,11,DeMarcus Cousins,28
2885,19,Julius Randle,28
3497,21,Giannis Antetokounmpo,27
2450,18,Giannis Antetokounmpo,26
834,13,James Harden,25


# "The Good Ol' Hockey Game, is the Best Game You Can Name" Award (presented by Dominik Hasek)*

most goaltends committed (credit to Kdog122025 for the idea)

In [22]:
goaltends=pbp[pbp['HOMEDESCRIPTION'].str.contains('Goaltending',na=False)|
        pbp['VISITORDESCRIPTION'].str.contains('Goaltending',na=False)]

In [23]:
goaltends.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
414,Dwight Howard,527
1207,Samuel Dalembert,351
576,JaVale McGee,279
52,Andre Drummond,214
697,Joel Przybilla,171
333,DeAndre Jordan,167
1226,Serge Ibaka,166
810,Kevin Garnett,165
1234,Shaquille O'Neal,149
802,Kenyon Martin,139


In [24]:
goaltends.groupby(['season','PLAYER1_NAME']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,season,PLAYER1_NAME,count
1489,8,Dwight Howard,58
2075,11,Dwight Howard,57
2100,11,JaVale McGee,56
1686,9,Dwight Howard,53
1394,7,Samuel Dalembert,48
763,4,Samuel Dalembert,44
1876,10,Dwight Howard,44
1593,8,Samuel Dalembert,43
2493,13,Dwight Howard,43
899,5,Joel Przybilla,41


# The Most 3-Pt Shooting Fouls Committed Award*

most 3-point shooting fouls committed (credit to watchingsongsDL, kingcobweb & An-Indian-In-The-NBA for the idea)

In [25]:
third_free_throws=pbp[pbp['HOMEDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)|
                          pbp['VISITORDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)]

shooting_fouls=pbp[pbp['HOMEDESCRIPTION'].str.contains('S.FOUL',na=False)|
        pbp['VISITORDESCRIPTION'].str.contains('S.FOUL',na=False)]

fouls_on_missed_threes=shooting_fouls.merge(right=third_free_throws,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

three_pointers=pbp[pbp['HOMEDESCRIPTION'].str.contains('3PT',na=False)|
        pbp['VISITORDESCRIPTION'].str.contains('3PT',na=False)]

made_threes=three_pointers[~three_pointers.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~three_pointers.VISITORDESCRIPTION.str.startswith('MISS',na=False)]

fouls_on_made_threes=shooting_fouls.merge(right=made_threes,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

fouls_on_threes=fouls_on_missed_threes.append(fouls_on_made_threes)

In [26]:
fouls_on_threes.groupby('PLAYER1_NAME_x').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME_x,count
712,Jeff Teague,59
893,Kobe Bryant,58
616,JR Smith,57
1175,Paul George,57
207,Carmelo Anthony,56
344,Darren Collison,55
809,Josh Richardson,55
1363,Stephen Curry,55
450,Dwyane Wade,53
105,Avery Bradley,52


In [27]:
fouls_on_threes.groupby(['season_x','PLAYER1_NAME_x']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,season_x,PLAYER1_NAME_x,count
4182,19,Josh Hart,14
3813,18,Josh Richardson,13
4064,19,De'Aaron Fox,13
4577,20,Kent Bazemore,13
3585,17,Stephen Curry,12
4067,19,DeAndre' Bembry,12
4615,20,Matisse Thybulle,12
4647,20,Patrick Beverley,12
3357,17,Damian Lillard,11
3465,17,Jrue Holiday,11


# The "Master Baiter" Award (sponsored by Bass Pro Shops & Kleenex)

most 3-point shooting fouls drawn (differs from PBPStats)

In [28]:
fouls_on_threes.groupby('PLAYER1_NAME_y').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME_y,count
433,James Harden,547
426,Jamal Crawford,243
623,Lou Williams,242
875,Stephen Curry,212
200,Damian Lillard,206
405,JJ Redick,190
566,Kevin Durant,161
554,Kemba Walker,141
596,Kyle Lowry,141
585,Kobe Bryant,139


In [29]:
fouls_on_threes.groupby(['season_y','PLAYER1_NAME_y']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,season_y,PLAYER1_NAME_y,count
2149,17,James Harden,121
2592,19,James Harden,95
2816,20,James Harden,70
2360,18,James Harden,69
2190,17,Lou Williams,54
1805,15,Lou Williams,43
1952,16,James Harden,43
1991,16,Lou Williams,43
2928,20,Trae Young,43
2762,20,Damian Lillard,40


# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)

Most charges drawn per 36 minutes (minimum 500 MP and 12 MP/G for career, minimum 70 percent of games played for season), credit to morron88 for the idea to separate charges & loose balls in 2020

In [30]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2013-14",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']].\
assign(season="2013-14")
standings_seasons=["2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21","2021-22"]
for seas in standings_seasons:
    compact_standings=compact_standings.append(leaguestandings.LeagueStandings(league_id='00',
                                             season=seas,season_type="Regular Season").\
                                               standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']].\
                        assign(season=seas))
    print(seas)
    time.sleep(5)

compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21
2021-22


In [31]:
#from 2015-2016 playoff to current
hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="Totals",season="2016-17",season_type_all_star="Regular Season").hustle_stats_player.get_data_frame().\
assign(season="2016-17")
hustle_seasons=["2017-18","2018-19","2019-20","2020-21","2021-22"]
for seas in hustle_seasons:
    hustle=hustle.append(leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="Totals",season=seas,season_type_all_star="Regular Season").hustle_stats_player.get_data_frame().\
                        assign(season=seas))
    print(seas)
    time.sleep(5)

2017-18
2018-19
2019-20
2020-21
2021-22


In [32]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on=['TEAM_ID','season'],right_on=['TeamID','season'])
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [33]:
hustle_totals=hustle.groupby(['PLAYER_ID','PLAYER_NAME'])[['G','MIN','CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].sum().reset_index()
hustle_totals[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_totals[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_totals.MIN,axis=0).multiply(36,axis=0)

In [34]:
hustle_totals.query('MIN >= 500 & MIN/G >= 12').nlargest(10,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','G','MIN','CHARGES_DRAWN']]

,PLAYER_NAME,G,MIN,CHARGES_DRAWN
119,Marreese Speights,134,1961.0,0.972973
61,Ersan Ilyasova,298,6237.0,0.750361
49,Andrew Bogut,62,933.0,0.694534
728,Moritz Wagner,196,2963.0,0.692541
267,Quincy Acy,118,2040.0,0.564706
283,Aron Baynes,302,5383.0,0.508267
219,Kenneth Faried,130,2485.0,0.507042
279,Willie Reed,113,1459.0,0.493489
550,Ryan Arcidiacono,217,3721.0,0.483741
881,Garrison Mathews,147,2977.0,0.483708


In [35]:
hustle_70percent_gp.nlargest(10,columns='CHARGES_DRAWN',keep='all')[['season','G','MIN','PLAYER_NAME','CHARGES_DRAWN']]

,season,G,MIN,PLAYER_NAME,CHARGES_DRAWN
1187,2018-19,67,1231.0,Ersan Ilyasova,1.432981
317,2016-17,82,1286.0,Marreese Speights,0.951788
406,2016-17,73,905.0,Salah Mejri,0.875138
1996,2019-20,58,930.0,Ryan Arcidiacono,0.735484
538,2017-18,64,576.0,Brandon Paul,0.687500
644,2017-18,69,1730.0,Ersan Ilyasova,0.665896
1681,2019-20,68,1677.0,Derrick White,0.644007
1936,2019-20,63,1749.0,Montrezl Harrell,0.617496
146,2016-17,82,2142.0,Ersan Ilyasova,0.605042
1874,2019-20,58,2098.0,Kyle Lowry,0.583413


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes (minimum 500 MP and 12 MP/G for career, minimum 70 percent of games played for season)

In [36]:
hustle_totals.query('MIN >= 500 & MIN/G >= 12').nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','G','MIN','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,G,MIN,LOOSE_BALLS_RECOVERED
563,Shaquille Harrison,175,2653.0,1.967584
750,Ky Bowman,45,1015.0,1.773399
195,Jeremy Lin,111,2344.0,1.766212
304,Michael Carter-Williams,201,3686.0,1.728703
474,Ben Simmons,275,9326.0,1.710058
451,Treveon Graham,180,2883.0,1.660770
773,Johnathan Williams,39,552.0,1.630435
159,Patrick Beverley,302,8180.0,1.628362
148,Ty Lawson,69,1732.0,1.600462
406,T.J. McConnell,400,9084.0,1.585205


In [37]:
hustle_70percent_gp.nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['season','G','MIN','PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,season,G,MIN,PLAYER_NAME,LOOSE_BALLS_RECOVERED
931,2017-18,80,2914.0,Russell Westbrook,2.112560
601,2017-18,81,1743.0,Dejounte Murray,2.106713
2572,2020-21,57,551.0,Thanasis Antetokounmpo,2.090744
1433,2018-19,77,2841.0,Paul George,2.027455
1472,2018-19,73,1430.0,Shaquille Harrison,2.013986
564,2017-18,72,1209.0,Corey Brewer,1.995037
1328,2018-19,67,1643.0,Kent Bazemore,1.993914
636,2017-18,63,1809.0,Elfrid Payton,1.950249
526,2017-18,81,2732.0,Ben Simmons,1.950220
366,2016-17,67,2058.0,Patrick Beverley,1.941691


# The Plexiglass Award

most deflections per 36 minutes (minimum 500 MP and 12 MP/G for career, minimum 70 percent of games played for season)

In [38]:
hustle_totals.query('MIN >= 500 & MIN/G >= 12').nlargest(10,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','G','MIN','DEFLECTIONS']]

,PLAYER_NAME,G,MIN,DEFLECTIONS
863,Matisse Thybulle,196,4270.0,5.201874
516,Gary Payton II,142,2056.0,5.060311
1064,Jose Alvarado,54,834.0,4.705036
284,Nerlens Noel,308,5811.0,4.677336
709,De'Anthony Melton,235,4853.0,4.606635
481,Kris Dunn,245,5897.0,4.358827
309,Robert Covington,393,12173.0,4.258605
987,Facundo Campazzo,130,2609.0,4.194711
43,Tony Allen,93,2187.0,4.164609
406,T.J. McConnell,400,9084.0,4.149273


In [39]:
hustle_70percent_gp.nlargest(10,columns='DEFLECTIONS',keep='all')[['season','G','MIN','PLAYER_NAME','DEFLECTIONS']]

,season,G,MIN,PLAYER_NAME,DEFLECTIONS
2445,2020-21,65,1298.0,Matisse Thybulle,5.602465
1866,2019-20,51,1269.0,Kris Dunn,5.361702
3033,2021-22,66,1685.0,Matisse Thybulle,5.298516
2825,2021-22,71,1248.0,Gary Payton II,5.192308
1410,2018-19,77,1056.0,Nerlens Noel,5.079545
1945,2019-20,61,1127.0,Nerlens Noel,5.078971
2560,2020-21,69,1796.0,T.J. McConnell,4.910913
392,2016-17,67,2119.0,Robert Covington,4.807928
1664,2019-20,60,1167.0,De'Anthony Melton,4.781491
429,2016-17,81,2133.0,T.J. McConnell,4.708861


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes (minimum 500 MP and 12 MP/G for career, minimum 70 percent of games played for season)

In [40]:
hustle_totals.query('MIN >= 500 & MIN/G >= 12').nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','G','MIN','SCREEN_AST_PTS']]

,PLAYER_NAME,G,MIN,SCREEN_AST_PTS
120,Roy Hibbert,48,682.0,16.944282
310,Rudy Gobert,423,13777.0,16.274080
28,Zaza Pachulia,207,3118.0,15.425273
65,Marcin Gortat,211,5380.0,15.417100
293,Cody Zeller,277,6296.0,15.072427
493,Jakob Poeltl,416,8409.0,14.684267
321,Lucas Nogueira,106,1506.0,14.605578
387,Jusuf Nurkic,317,8164.0,13.863792
178,Ed Davis,287,4509.0,13.860279
204,Tristan Thompson,342,8511.0,13.687698


In [41]:
hustle_70percent_gp.nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['season','G','MIN','PLAYER_NAME','SCREEN_AST_PTS']]

,season,G,MIN,PLAYER_NAME,SCREEN_AST_PTS
79,2016-17,62,1725.0,Cody Zeller,17.697391
2535,2020-21,71,2187.0,Rudy Gobert,17.646091
3124,2021-22,66,2120.0,Rudy Gobert,17.167925
1755,2019-20,72,1326.0,Ivica Zubac,16.805430
1992,2019-20,68,2333.0,Rudy Gobert,16.804115
2237,2020-21,58,966.0,Dwight Powell,16.695652
1024,2017-18,69,972.0,Zaza Pachulia,16.629630
1769,2019-20,66,1171.0,Jakob Poeltl,16.601196
3153,2021-22,76,1999.0,Steven Adams,16.442221
485,2016-17,70,1268.0,Zaza Pachulia,16.296530


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 500 MP and 12 MP/G for career, minimum 50 percent of games played for season)

In [42]:
# traditional defensive stats approximate by tracking
# (deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
# defensive boxouts and defensive loose balls recovered not recorded in 2016-17
hustle_start_2018=hustle.query('season != "2016-17"')
hustle_totals_start_2018=hustle.\
groupby(['PLAYER_ID','PLAYER_NAME'])[['G','MIN','CHARGES_DRAWN', 'CONTESTED_SHOTS_2PT','CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
sum().reset_index()

hustle_totals_start_2018_qualify=hustle_totals_start_2018.query('MIN >= 500 & MIN/G >= 12')

per_36_percent_ranks_career=hustle_totals_start_2018_qualify[['CHARGES_DRAWN', 'CONTESTED_SHOTS_2PT','CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_totals_start_2018_qualify.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks_career["sum"]=per_36_percent_ranks_career.sum(axis=1)
hustle_totals_start_2018_ranks=hustle_totals_start_2018_qualify.\
merge(per_36_percent_ranks_career,how='left',left_index=True,right_index=True)

In [43]:
hustle_totals_start_2018_ranks.nsmallest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|MIN|sum|pct_rank$',axis=1)

,PLAYER_NAME,MIN,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
7,Jamal Crawford,5027.0,0.187888,0.007764,0.007764,0.327640,0.090062,0.186335,0.807453
135,Anthony Morrow,714.0,0.078416,0.034161,0.069876,0.593168,0.021739,0.023292,0.820652
13,Tony Parker,3661.0,0.218944,0.055901,0.145963,0.090062,0.218944,0.231366,0.961180
454,Keifer Sykes,566.0,0.078416,0.049689,0.043478,0.532609,0.063665,0.223602,0.991460
25,Boris Diaw,1283.0,0.078416,0.692547,0.138199,0.051242,0.021739,0.023292,1.005435
78,Sergio Rodriguez,1517.0,0.349379,0.068323,0.034161,0.537267,0.021739,0.023292,1.034161
160,Marcus Thornton,575.0,0.078416,0.015528,0.369565,0.572981,0.021739,0.023292,1.081522
90,Spencer Hawes,797.0,0.078416,0.855590,0.135093,0.063665,0.021739,0.023292,1.177795
103,Ramon Sessions,1203.0,0.078416,0.091615,0.450311,0.451863,0.049689,0.083851,1.205745
548,Mindaugas Kuzminskas,1018.0,0.078416,0.340062,0.343168,0.414596,0.021739,0.023292,1.221273


In [44]:
hustle_totals_start_2018_ranks[hustle_totals_start_2018_ranks['PLAYER_NAME'].isin(['Patrick Beverley','Russell Westbrook'])].\
filter(regex='PLAYER_NAME|MIN|sum|pct_rank$',axis=1)

,PLAYER_NAME,MIN,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
110,Russell Westbrook,15441.0,0.652174,0.026398,0.024845,0.816770,0.153727,0.860248,2.534161
159,Patrick Beverley,8180.0,0.947205,0.265528,0.279503,0.899068,0.094720,0.888199,3.374224


In [45]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').query('season != "2016-17"').copy()
per_36_percent_ranks_seas=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS_2PT','CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks_seas["sum"]=per_36_percent_ranks_seas.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks_seas,how='left',left_index=True,right_index=True)

In [46]:
hustle_50percent_gp_ranks.nsmallest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|season|MIN|sum|pct_rank$',axis=1)

,PLAYER_NAME,MIN,season,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
2432,Malik Monk,878.0,2020-21,0.150317,0.046740,0.128679,0.175418,0.036930,0.141373,0.679458
2140,Bryn Forbes,1354.0,2020-21,0.150317,0.006924,0.201385,0.066359,0.173110,0.098096,0.696192
3162,Terrence Ross,1448.0,2021-22,0.150317,0.055395,0.109059,0.295730,0.051933,0.039238,0.701673
2682,Bryn Forbes,1286.0,2021-22,0.150317,0.007501,0.010964,0.070398,0.273514,0.189267,0.701962
2182,Damyean Dotson,907.0,2020-21,0.150317,0.021350,0.276976,0.073283,0.100981,0.085401,0.708309
2803,Eric Gordon,1670.0,2021-22,0.381997,0.038084,0.004039,0.103289,0.135026,0.047317,0.709752
2085,Aaron Gordon,1384.0,2020-21,0.150317,0.276976,0.017888,0.165609,0.032314,0.090017,0.733122
3216,Will Barton,2276.0,2021-22,0.150317,0.130410,0.180612,0.241200,0.002308,0.047894,0.752741
2726,D.J. Augustin,883.0,2021-22,0.508367,0.001731,0.022504,0.069244,0.046740,0.137046,0.785632
2916,Joe Ingles,1122.0,2021-22,0.150317,0.015580,0.142527,0.231391,0.253895,0.009233,0.802943


# The "Got that Dawg in Him" Award (presented by Air Bud)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 500 MP and 12 MP/G for career, minimum 50 percent of games played for season) (credit to memeticengineering for the idea)

In [47]:
hustle_totals_start_2018_ranks.nlargest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|MIN|sum|pct_rank$',axis=1)

,PLAYER_NAME,MIN,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
600,Jordan Bell,1937.0,0.855590,0.905280,0.989130,0.739130,0.906832,0.900621,5.296584
91,Thaddeus Young,11421.0,0.937888,0.704969,0.981366,0.962733,0.759317,0.784161,5.130435
266,Draymond Green,11356.0,0.883540,0.793478,0.959627,0.958075,0.889752,0.593168,5.077640
991,Freddie Gillespie,511.0,0.897516,0.920807,0.586957,0.866460,0.701863,0.987578,4.961180
728,Moritz Wagner,2963.0,0.995342,0.757764,0.827640,0.692547,0.927019,0.641304,4.841615
750,Ky Bowman,1015.0,0.871118,0.534161,0.911491,0.965839,0.565217,0.993789,4.841615
57,Ian Mahinmi,3006.0,0.874224,0.940994,0.293478,0.939441,0.998447,0.708075,4.754658
242,Anthony Davis,11982.0,0.805901,0.815217,0.871118,0.754658,0.673913,0.795031,4.715839
16,Nene,2502.0,0.905280,0.819876,0.737578,0.897516,0.940994,0.381988,4.683230
300,Kelly Olynyk,9190.0,0.950311,0.760870,0.855590,0.768634,0.821429,0.520186,4.677019


In [48]:
hustle_50percent_gp_ranks.nlargest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|season|sum|pct_rank$',axis=1)

,PLAYER_NAME,season,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
1166,Draymond Green,2018-19,0.874207,0.783612,0.983843,0.952683,0.949221,0.912868,5.456434
964,Thaddeus Young,2017-18,0.874784,0.719561,0.975765,0.984420,0.834391,0.971148,5.360069
875,Nene,2017-18,0.716099,0.815349,0.932487,0.926140,0.986728,0.896134,5.272937
1209,Gorgui Dieng,2018-19,0.889209,0.927294,0.773226,0.870167,0.877669,0.903635,5.241200
664,Gorgui Dieng,2017-18,0.965378,0.884593,0.701096,0.832660,0.922677,0.900173,5.206578
630,Dwight Powell,2017-18,0.828044,0.845355,0.979227,0.820542,0.897288,0.832949,5.203405
1495,Thaddeus Young,2018-19,0.956722,0.673399,0.971148,0.963070,0.802654,0.727640,5.094634
1298,Jordan Bell,2018-19,0.905943,0.923832,0.945182,0.634737,0.860935,0.735141,5.005770
1401,Mitchell Robinson,2018-19,0.669937,0.970571,0.991922,0.691864,0.909983,0.739469,4.973745
1739,Gorgui Dieng,2019-20,0.980958,0.912291,0.557992,0.983266,0.828044,0.692441,4.954991


In [49]:
hustle_50percent_gp_ranks.to_csv(path_or_buf="Output Data/Seasonal Hustle Ranks.csv")
hustle_totals_start_2018_ranks.to_csv(path_or_buf="Output Data/Career Hustle Ranks.csv")

# The "David vs Goliath" Award*

most shots blocked with at least a 5-inch difference in height

In [50]:
player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats(season="1996-97").league_dash_player_bio_stats.get_data_frame()
player_bio["season"]=97
for seas in season_slug:
    new_player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=seas).league_dash_player_bio_stats.get_data_frame()
    new_player_bio["season"]=int(seas[-2:])
    player_bio=player_bio.append(new_player_bio)
    print(seas)
    time.sleep(5)

1997-98
1998-99
1999-00
2000-01
2001-02
2002-03
2003-04
2004-05
2005-06
2006-07
2007-08
2008-09
2009-10
2010-11
2011-12
2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21
2021-22


In [51]:
blocks=pbp[pbp['HOMEDESCRIPTION'].str.contains('BLOCK',na=False)|
                          pbp['VISITORDESCRIPTION'].str.contains('BLOCK',na=False)]

blocking_player_heights=blocks.merge(player_bio[['PLAYER_ID','season','PLAYER_HEIGHT_INCHES']],
                                     left_on=['PLAYER1_ID','season'],
                                     right_on=['PLAYER_ID','season']).\
merge(player_bio[['PLAYER_ID','season','PLAYER_HEIGHT_INCHES']],
      left_on=['PLAYER3_ID','season'],
      right_on=['PLAYER_ID','season'])
blocking_player_heights["height_difference"]=blocking_player_heights['PLAYER_HEIGHT_INCHES_x']-blocking_player_heights['PLAYER_HEIGHT_INCHES_y']

In [52]:
blocking_player_heights.query('height_difference >= 5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
399,Dwyane Wade,261
770,Kyle Lowry,181
422,Eric Bledsoe,132
720,Kemba Walker,128
1097,Russell Westbrook,119
989,Patrick Beverley,117
95,Baron Davis,116
562,James Harden,114
224,Chris Paul,105
654,John Wall,104


In [53]:
blocking_player_heights.query('height_difference >= 5').groupby(['season','PLAYER3_NAME']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,season,PLAYER3_NAME,count
1573,9,Dwyane Wade,36
1936,11,Dwyane Wade,30
4196,22,Patrick Beverley,28
3836,21,Fred VanVleet,26
809,5,Dwyane Wade,25
4088,22,Fred VanVleet,25
1767,10,Dwyane Wade,23
2269,13,Eric Bledsoe,23
2838,16,Kyle Lowry,23
2106,12,Dwyane Wade,21


# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award

Lowest contested rebound percentage (minimum 250 rebounds for career, 50% games played for season)

In [54]:
rebounding=playerdashptreb.PlayerDashPtReb(season="2013-14",team_id=0,player_id=0).overall_rebounding.get_data_frame().assign(season="2013-14")
rebounding_seas=["2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21","2021-22"]
for seas in rebounding_seas:
    rebounding=rebounding.append(playerdashptreb.PlayerDashPtReb(season=seas,team_id=0,player_id=0).overall_rebounding.get_data_frame().\
                                 assign(season=seas))
    print(seas)
    time.sleep(5)

2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21
2021-22


In [55]:
reb_totals=rebounding.groupby(['PLAYER_ID','PLAYER_NAME_LAST_FIRST'])[['REB','C_REB','UC_REB']].sum().reset_index()
reb_totals["C_REB_PCT"]=reb_totals.C_REB/(reb_totals.REB)
reb_totals["UC_REB_PCT"]=reb_totals.UC_REB/(reb_totals.REB)

In [56]:
reb_totals.query('REB>=250').nsmallest(10,columns='C_REB_PCT',keep='all')

,PLAYER_ID,PLAYER_NAME_LAST_FIRST,REB,C_REB,UC_REB,C_REB_PCT,UC_REB_PCT
622,1626145,"Jones, Tyus",794,71,723,0.089421,0.910579
441,203148,"Roberts, Brian",327,31,296,0.094801,0.905199
119,101127,"Jack, Jarrett",795,82,713,0.103145,0.896855
19,1891,"Terry, Jason",389,41,348,0.105398,0.894602
39,2225,"Parker, Tony",752,80,672,0.106383,0.893617
640,1626166,"Payne, Cameron",616,66,550,0.107143,0.892857
916,1628988,"Holiday, Aaron",433,48,385,0.110855,0.889145
145,200755,"Redick, JJ",1117,124,993,0.111012,0.888988
162,200826,"Barea, J.J.",873,97,776,0.111111,0.888889
200,201571,"Augustin, D.J.",1153,130,1023,0.112749,0.887251


In [57]:
reb_totals[reb_totals.PLAYER_NAME_LAST_FIRST=="Anthony, Carmelo"]

,PLAYER_ID,PLAYER_NAME_LAST_FIRST,REB,C_REB,UC_REB,C_REB_PCT,UC_REB_PCT
59,2546,"Anthony, Carmelo",3207,961,2246,0.299657,0.700343


In [58]:
games_percentages=hustle_w_gp_qualify.copy()[['season','PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on=['season','PLAYER_ID'],right_on=['season','PLAYER_ID']).query('G_PERCENT >= 0.5')

In [59]:
reb_w_gp_qualify.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT','season']]

,PLAYER_NAME,C_REB_PCT,season
1622,Allen Crabbe,0.053,2019-20
2245,Patty Mills,0.060,2020-21
3004,D.J. Augustin,0.060,2021-22
548,Raul Neto,0.061,2017-18
811,Joe Young,0.063,2017-18
740,John Wall,0.068,2017-18
816,Jarrett Jack,0.071,2017-18
669,Jamal Crawford,0.072,2017-18
1072,Tyus Jones,0.074,2018-19
2623,Tyus Jones,0.076,2021-22


# The "Where There's a Will, There's a Way" Award (presented by Dennis Rodman)

Highest contested rebound percentage (minimum 250 rebounds for career, 50% games played for season)

In [60]:
reb_totals.query('REB>=250').nlargest(10,columns='C_REB_PCT',keep='all')

,PLAYER_ID,PLAYER_NAME_LAST_FIRST,REB,C_REB,UC_REB,C_REB_PCT,UC_REB_PCT
206,201577,"Lopez, Robin",3296,1917,1379,0.581614,0.418386
935,1629011,"Robinson, Mitchell",1700,972,728,0.571765,0.428235
409,203105,"Ezeli, Festus",402,224,178,0.557214,0.442786
711,1627751,"Poeltl, Jakob",2499,1391,1108,0.556623,0.443377
297,202336,"Sanders, Larry",323,177,146,0.547988,0.452012
246,201946,"Hansbrough, Tyler",624,338,286,0.541667,0.458333
220,201593,"Pekovic, Nikola",699,373,326,0.533619,0.466381
132,101161,"Johnson, Amir",2259,1195,1064,0.528995,0.471005
221,201595,"Dorsey, Joey",276,146,130,0.528986,0.471014
401,203096,"Sullinger, Jared",1721,891,830,0.517722,0.482278


In [61]:
reb_w_gp_qualify.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT','season']]

,PLAYER_NAME,C_REB_PCT,season
2216,Robin Lopez,0.605,2020-21
2862,Mitchell Robinson,0.605,2021-22
281,Robin Lopez,0.600,2016-17
1301,Robin Lopez,0.594,2018-19
2747,Jakob Poeltl,0.578,2021-22
504,Jakob Poeltl,0.577,2017-18
2212,Daniel Gafford,0.566,2020-21
2914,Jock Landale,0.565,2021-22
2243,Jakob Poeltl,0.564,2020-21
1404,Amir Johnson,0.559,2018-19
